# Reproducing table 6

This notebook currently imports results from a run of PHC.py and attempts to map them to Table 6.

Table 6 results are from Shoaib M, Ramamohan V. **Simulation modeling and analysis of primary health center operations**. *SIMULATION* 98(3):183-208. (2022). <https://doi.org/10.1177/00375497211030931>.

## Set up

In [1]:
# To run model
import PHC
import numpy as np

# To import and process results
import pandas as pd
import xlrd
import os

# To speed up run time
from multiprocessing import Pool

# Additional package to record runtime of this notebook
import time
start = time.time()

In [2]:
table6_path = '../original_study/tab6.csv'

## Run model

Set parameters for each configuration based on Table 3.

In [3]:
# For all: days 365, warm up 180, 6 inpatient beds, 1 delivery
# bed, 3 staff nurse, 1 NCD nurse. These are default parameters in the model,
# and below, have just specified the parameters that changed between
# configurations

t6_c1_param = {
    'doc_cap': 2,
    'OPD_iat': 4,
    'IPD_iat': 2880,
    'delivery_iat': 1440,
    'ANC_iat': 1440,
    'rep_file': 't6_c1.xls',
    'replication': 100
}

t6_c2_param = {
    'doc_cap': 1,
    'OPD_iat': 9,
    'IPD_iat': 2880,
    'delivery_iat': 2880,
    'ANC_iat': 2880,
    'rep_file': 't6_c2.xls',
    'replication': 100
}

t6_c3_param = {
    'doc_cap': 1,
    'OPD_iat': 9,
    'IPD_iat': 2880,
    'any_delivery': False,
    'any_ANC': False,
    'rep_file': 't6_c3.xls',
    'replication': 100
}

t6_c4_param = {
    'doc_cap': 2,
    'OPD_iat': 3,
    'mean': 5,
    'sd': 1,
    'consult_boundary_1': 2,
    'consult_boundary_2': 2,
    'IPD_iat': 2880,
    'delivery_iat': 1440,
    'ANC_iat': 1440,
    'rep_file': 't6_c4.xls',
    'replication': 100
}

In [4]:
# Create list of parameter dictionaries
config = [t6_c1_param, t6_c2_param, t6_c3_param, t6_c4_param]

# Append 's_' to all items
for i, d in enumerate(config):
    config[i] = {f's_{k}': v for k, v in d.items()}

Run model for each configuration using parallel processing.

In [5]:
# Wrapper function to allow input of dictionary with pool
def wrapper(d):
    return PHC.main(**d)

# Create a process pool that uses all CPUs
with Pool(4) as pool:
    # Run PHC.main() using each of inputs from config
    pool.map(wrapper, config)

 No of replications done 0
 No of replications done 0
 No of replications done 1
 No of replications done 1
 No of replications done 0
 No of replications done 2
 No of replications done 0
 No of replications done 2
 No of replications done 3
 No of replications done 3
 No of replications done 1
 No of replications done 4
 No of replications done 4
 No of replications done 5
 No of replications done 1
 No of replications done 5
 No of replications done 6
 No of replications done 2
 No of replications done 6
 No of replications done 7
 No of replications done 8
 No of replications done 7
 No of replications done 2
 No of replications done 3
 No of replications done 9
 No of replications done 8
 No of replications done 10
 No of replications done 9
 No of replications done 11
 No of replications done 4
 No of replications done 10
 No of replications done 12
 No of replications done 3
 No of replications done 11
 No of replications done 13
 No of replications done 14
 No of replications d

## Import and process replication results

In [7]:
# Make dictionary with labels from table 6, and corresponding names from model output
t6_labels = {
  'doc occ': 'Doctor utilisation',
  'NCD occ': 'NCD Nurse utilisation',
  'staff nurse occ': 'Staff nurse utilisation',
  'pharm occ': 'Pharmacist utilisation',
  'lab occ': 'Lab utilisation',
  'ipd bed occ': 'Inpatient bed utilisation',
  'del occ': 'Labour bed utilisation',  # "Del" stands for delivery
  'opd q len': 'Mean length of OPD queue (number of patients)',
  'OPD Q wt': 'OPD queue waiting time (minutes)',
  'pharmacy q len': 'Mean length of pharmacy queue (number of patients)',
  'Pharmacy Q wt': 'Pharmacy queue waiting time (minutes)',
  'lab q len': 'Mean length of Lab queue (number of patients)',
  'Lab Q wt': 'Lab queue waiting time (minutes)',
  'prop_del_referred': 'Fraction of childbirth cases referred'
}

In [8]:
# List of files to loop through
files = ['t6_c1', 't6_c2', 't6_c3', 't6_c4']

# Empty list to store results
result_list = []

for f in files:
    # Import .xls and convert to pandas dataframe
    book = xlrd.open_workbook(os.path.join('outputs', f'{f}.xls'))
    result = pd.read_excel(book, header=None, index_col=0)

    # Add proportion of childbirth cases referred
    result.loc['prop_del_referred'] = (
        result.loc['del referred'] / result.loc['Del patients'])

    # Find mean and standard deviation from the replication
    # Save as dataframe, dropping the duplicate rows (NCD occ twice)
    res = pd.DataFrame({
        f'model_{f}_mean': result.mean(axis=1),
        f'model_{f}_sd': result.std(axis=1)
    }).drop_duplicates()

    # Save to list
    result_list.append(res)

In [9]:
# Combine into single dataframe
summary = (pd.concat(result_list, axis=1)
           .reset_index()
           .rename(columns= {0: 'model_outcome'}))

# Add labels to model results
summary['t6_outcome'] = summary['model_outcome'].map(t6_labels)

summary

,model_outcome,model_t6_c1_mean,model_t6_c1_sd,model_t6_c2_mean,model_t6_c2_sd,model_t6_c3_mean,model_t6_c3_sd,model_t6_c4_mean,model_t6_c4_sd,t6_outcome
0,OPD patients,3.312637e+04,1.785897e+02,14890.180000,110.461052,14857.250000,109.384816,4.405896e+04,2.118088e+02,NaN
1,IPD patients,1.819400e+02,1.219026e+01,181.990000,13.949762,183.540000,14.546630,1.843800e+02,1.455382e+01,NaN
2,ANC patients,3.643800e+02,1.464377e+01,211.900000,11.898986,0.000000,0.000000,3.676700e+02,1.572494e+01,NaN
3,Del patients,3.654500e+02,1.906574e+01,181.470000,14.669046,NaN,NaN,3.664800e+02,2.039978e+01,NaN
4,OPD Q wt,8.811218e-03,3.391605e-03,0.182605,0.029487,0.034597,0.001101,6.968764e+00,2.516115e-01,OPD queue waiting time (minutes)
5,Pharmacy Q wt,1.025055e+00,2.226311e-02,0.244612,0.008553,0.231061,0.006068,1.284367e+00,1.971490e-02,Pharmacy queue waiting time (minutes)
6,Lab Q wt,2.076878e+00,5.626489e-02,0.604793,0.022624,0.569570,0.018696,3.152544e+00,7.267482e-02,Lab queue waiting time (minutes)
7,doc occ,2.686463e-01,2.152930e-03,0.372444,0.003851,0.353664,0.002500,1.143672e+00,5.670042e-03,Doctor utilisation
8,Lab patient list,1.747950e+06,1.003715e+06,780873.910000,449025.249681,782899.560000,450041.784075,2.313950e+06,1.329765e+06,NaN
9,OPD q len,8.397417e-03,4.244432e-03,0.177137,0.036704,0.034150,0.001157,6.842035e+00,2.987361e-01,NaN


## Import table 6 results and compare against run results

In [10]:
# Import table 6
t6 = pd.read_csv(table6_path).rename(columns={'outcome': 't6_outcome'})
t6

,t6_outcome,config1_mean,config1_sd,config2_mean,config2_sd,config3_mean,config3_sd,benchmark_mean,benchmark_sd
0,Doctor utilisation,0.268,0.003,0.372,0.004,0.354,0.002,1.142,0.006
1,NCD Nurse utilisation,0.865,0.011,0.469,0.005,0.468,0.005,1.232,0.019
2,Staff nurse utilisation,0.323,0.008,0.243,0.006,0.160,0.001,0.322,0.008
3,Pharmacist utilisation,0.643,0.004,0.288,0.003,0.289,0.003,0.855,0.005
4,Lab utilisation,0.559,0.008,0.254,0.004,0.239,0.004,0.736,0.011
5,Inpatient bed utilisation,0.093,0.004,0.055,0.003,0.011,0.001,0.093,0.004
6,Labour bed utilisation,0.283,0.010,0.153,0.009,NaN,NaN,0.281,0.012
7,Mean length of OPD queue (number of patients),0.000,0.000,0.007,0.001,0.001,0.000,0.817,0.027
8,OPD queue waiting time (minutes),0.009,0.004,0.171,0.032,0.034,0.001,6.789,0.268
9,Mean length of pharmacy queue (number of patie...,0.090,0.002,0.010,0.001,0.009,0.000,0.150,0.002


In [11]:
# Merge
compare = t6.merge(summary)
compare.head()

,t6_outcome,config1_mean,config1_sd,config2_mean,config2_sd,config3_mean,config3_sd,benchmark_mean,benchmark_sd,model_outcome,model_t6_c1_mean,model_t6_c1_sd,model_t6_c2_mean,model_t6_c2_sd,model_t6_c3_mean,model_t6_c3_sd,model_t6_c4_mean,model_t6_c4_sd
0,Doctor utilisation,0.268,0.003,0.372,0.004,0.354,0.002,1.142,0.006,doc occ,0.268646,0.002153,0.372444,0.003851,0.353664,0.002500,1.143672,0.005670
1,NCD Nurse utilisation,0.865,0.011,0.469,0.005,0.468,0.005,1.232,0.019,NCD occ,0.866110,0.009541,0.468966,0.005556,0.467952,0.004912,1.231325,0.018736
2,Staff nurse utilisation,0.323,0.008,0.243,0.006,0.160,0.001,0.322,0.008,staff nurse occ,0.322897,0.007589,0.243063,0.005772,0.160844,0.001431,0.323955,0.007993
3,Pharmacist utilisation,0.643,0.004,0.288,0.003,0.289,0.003,0.855,0.005,pharm occ,0.642966,0.003611,0.288813,0.002505,0.288200,0.002420,0.855421,0.004399
4,Lab utilisation,0.559,0.008,0.254,0.004,0.239,0.004,0.736,0.011,lab occ,0.559089,0.007612,0.253639,0.004737,0.239393,0.004237,0.733738,0.011902


In [12]:
compare_col = [
    ('config1_mean', 'model_t6_c1_mean'),
    ('config1_sd', 'model_t6_c1_sd'),
    ('config2_mean', 'model_t6_c2_mean'),
    ('config2_sd', 'model_t6_c2_sd'),
    ('config3_mean', 'model_t6_c3_mean'),
    ('config3_sd', 'model_t6_c3_sd'),
    ('benchmark_mean', 'model_t6_c4_mean'),
    ('benchmark_sd', 'model_t6_c4_sd'),]

for col in compare_col:
    # Find difference between two columns
    compare[f'change_{col[1]}'] = abs(compare[col[1]] - compare[col[0]])
    # Find percent change between two columns
    subset = compare[list(col)]
    pct_change = subset.pct_change(axis=1).iloc[:, 1]*100
    compare[f'pct_change_{col[1]}'] = pct_change

# Display each of the results
for col in compare_col:
    # Set outcome as index, and get the two results plus percent change
    subset = compare.set_index('t6_outcome')[
        list(col) + [f'change_{col[1]}', f'pct_change_{col[1]}']]
    display(subset)

,config1_mean,model_t6_c1_mean,change_model_t6_c1_mean,pct_change_model_t6_c1_mean
t6_outcome,,,,
Doctor utilisation,0.268,0.268646,0.000646,0.241148
NCD Nurse utilisation,0.865,0.866110,0.001110,0.128378
Staff nurse utilisation,0.323,0.322897,0.000103,-0.031752
Pharmacist utilisation,0.643,0.642966,0.000034,-0.005363
Lab utilisation,0.559,0.559089,0.000089,0.015856
Inpatient bed utilisation,0.093,0.093701,0.000701,0.753367
Labour bed utilisation,0.283,0.282600,0.000400,-0.141343
Mean length of OPD queue (number of patients),0.000,0.000778,0.000778,inf
OPD queue waiting time (minutes),0.009,0.008811,0.000189,-2.097576


,config1_sd,model_t6_c1_sd,change_model_t6_c1_sd,pct_change_model_t6_c1_sd
t6_outcome,,,,
Doctor utilisation,0.003,0.002153,0.000847,-28.235658
NCD Nurse utilisation,0.011,0.009541,0.001459,-13.264782
Staff nurse utilisation,0.008,0.007589,0.000411,-5.134792
Pharmacist utilisation,0.004,0.003611,0.000389,-9.728468
Lab utilisation,0.008,0.007612,0.000388,-4.849327
Inpatient bed utilisation,0.004,0.004367,0.000367,9.186436
Labour bed utilisation,0.010,0.011774,0.001774,17.739655
Mean length of OPD queue (number of patients),0.000,0.000300,0.000300,inf
OPD queue waiting time (minutes),0.004,0.003392,0.000608,-15.209881


,config2_mean,model_t6_c2_mean,change_model_t6_c2_mean,pct_change_model_t6_c2_mean
t6_outcome,,,,
Doctor utilisation,0.372,0.372444,0.000444,0.119476
NCD Nurse utilisation,0.469,0.468966,0.000034,-0.007179
Staff nurse utilisation,0.243,0.243063,0.000063,0.026056
Pharmacist utilisation,0.288,0.288813,0.000813,0.282359
Lab utilisation,0.254,0.253639,0.000361,-0.142255
Inpatient bed utilisation,0.055,0.055842,0.000842,1.531615
Labour bed utilisation,0.153,0.152300,0.000700,-0.457516
Mean length of OPD queue (number of patients),0.007,0.007237,0.000237,3.383308
OPD queue waiting time (minutes),0.171,0.182605,0.011605,6.786332


,config2_sd,model_t6_c2_sd,change_model_t6_c2_sd,pct_change_model_t6_c2_sd
t6_outcome,,,,
Doctor utilisation,0.004,0.003851,0.000149,-3.714913
NCD Nurse utilisation,0.005,0.005556,0.000556,11.123890
Staff nurse utilisation,0.006,0.005772,0.000228,-3.797845
Pharmacist utilisation,0.003,0.002505,0.000495,-16.488884
Lab utilisation,0.004,0.004737,0.000737,18.434378
Inpatient bed utilisation,0.003,0.003817,0.000817,27.237694
Labour bed utilisation,0.009,0.010717,0.001717,19.080109
Mean length of OPD queue (number of patients),0.001,0.001175,0.000175,17.541779
OPD queue waiting time (minutes),0.032,0.029487,0.002513,-7.853583


,config3_mean,model_t6_c3_mean,change_model_t6_c3_mean,pct_change_model_t6_c3_mean
t6_outcome,,,,
Doctor utilisation,0.354,0.353664,0.000336,-0.094850
NCD Nurse utilisation,0.468,0.467952,0.000048,-0.010319
Staff nurse utilisation,0.160,0.160844,0.000844,0.527584
Pharmacist utilisation,0.289,0.288200,0.000800,-0.276720
Lab utilisation,0.239,0.239393,0.000393,0.164587
Inpatient bed utilisation,0.011,0.011606,0.000606,5.509558
Labour bed utilisation,NaN,NaN,NaN,NaN
Mean length of OPD queue (number of patients),0.001,0.001365,0.000365,36.492949
OPD queue waiting time (minutes),0.034,0.034597,0.000597,1.756292


,config3_sd,model_t6_c3_sd,change_model_t6_c3_sd,pct_change_model_t6_c3_sd
t6_outcome,,,,
Doctor utilisation,0.002,0.002500,0.000500,24.984286
NCD Nurse utilisation,0.005,0.004912,0.000088,-1.754243
Staff nurse utilisation,0.001,0.001431,0.000431,43.097811
Pharmacist utilisation,0.003,0.002420,0.000580,-19.317095
Lab utilisation,0.004,0.004237,0.000237,5.918305
Inpatient bed utilisation,0.001,0.000960,0.000040,-3.990474
Labour bed utilisation,NaN,NaN,NaN,NaN
Mean length of OPD queue (number of patients),0.000,0.000047,0.000047,inf
OPD queue waiting time (minutes),0.001,0.001101,0.000101,10.108310


,benchmark_mean,model_t6_c4_mean,change_model_t6_c4_mean,pct_change_model_t6_c4_mean
t6_outcome,,,,
Doctor utilisation,1.142,1.143672,0.001672,0.146391
NCD Nurse utilisation,1.232,1.231325,0.000675,-0.054810
Staff nurse utilisation,0.322,0.323955,0.001955,0.607169
Pharmacist utilisation,0.855,0.855421,0.000421,0.049258
Lab utilisation,0.736,0.733738,0.002262,-0.307275
Inpatient bed utilisation,0.093,0.093867,0.000867,0.931754
Labour bed utilisation,0.281,0.281500,0.000500,0.177936
Mean length of OPD queue (number of patients),0.817,0.817289,0.000289,0.035409
OPD queue waiting time (minutes),6.789,6.968764,0.179764,2.647874


,benchmark_sd,model_t6_c4_sd,change_model_t6_c4_sd,pct_change_model_t6_c4_sd
t6_outcome,,,,
Doctor utilisation,0.006,0.005670,0.000330,-5.499308
NCD Nurse utilisation,0.019,0.018736,0.000264,-1.388453
Staff nurse utilisation,0.008,0.007993,0.000007,-0.082284
Pharmacist utilisation,0.005,0.004399,0.000601,-12.026390
Lab utilisation,0.011,0.011902,0.000902,8.200349
Inpatient bed utilisation,0.004,0.004481,0.000481,12.035618
Labour bed utilisation,0.012,0.012822,0.000822,6.846833
Mean length of OPD queue (number of patients),0.027,0.031792,0.004792,17.748389
OPD queue waiting time (minutes),0.268,0.251612,0.016388,-6.115105


## Format model results like table 6, and save

In [13]:
compare_col = [
    ['Configuration 1', 'model_t6_c1_mean', 'model_t6_c1_sd'],
    ['Configuration 2', 'model_t6_c2_mean', 'model_t6_c2_sd'],
    ['Configuration 3', 'model_t6_c3_mean', 'model_t6_c3_sd'],
    ['Benchmark Case', 'model_t6_c4_mean', 'model_t6_c4_sd']]

# Combine each pair of columns into string as in Table 6
for col in compare_col:
    compare[col[0]] = (
        round(compare[col[1]], 3).astype(str) + ' (' +
        round(compare[col[2]], 3).astype(str) + ')')

# Set outcome name as index, and select those columns
formatted_model_res = (compare
                       .set_index('t6_outcome')
                       .rename_axis('Simulation Outcome')[
                           [item[0] for item in compare_col]])
formatted_model_res

,Configuration 1,Configuration 2,Configuration 3,Benchmark Case
Simulation Outcome,,,,
Doctor utilisation,0.269 (0.002),0.372 (0.004),0.354 (0.002),1.144 (0.006)
NCD Nurse utilisation,0.866 (0.01),0.469 (0.006),0.468 (0.005),1.231 (0.019)
Staff nurse utilisation,0.323 (0.008),0.243 (0.006),0.161 (0.001),0.324 (0.008)
Pharmacist utilisation,0.643 (0.004),0.289 (0.003),0.288 (0.002),0.855 (0.004)
Lab utilisation,0.559 (0.008),0.254 (0.005),0.239 (0.004),0.734 (0.012)
Inpatient bed utilisation,0.094 (0.004),0.056 (0.004),0.012 (0.001),0.094 (0.004)
Labour bed utilisation,0.283 (0.012),0.152 (0.011),nan (nan),0.281 (0.013)
Mean length of OPD queue (number of patients),0.001 (0.0),0.007 (0.001),0.001 (0.0),0.817 (0.032)
OPD queue waiting time (minutes),0.009 (0.003),0.183 (0.029),0.035 (0.001),6.969 (0.252)


In [14]:
# Find run time in seconds
end = time.time()
runtime = round(end-start)

# Display converted to minutes and seconds
print(f'Notebook run time: {runtime//60}m {runtime%60}s')

Notebook run time: 61m 47s
